# Demonstration of Proof-of-Work and Mining using Python

### Importing the SHA-256 hashing algorithm

In [1]:
from hashlib import sha256

### Defining the function that will output the data using the hashing algorithm

Hashing algorithms are the backbone of the security in the blockchain. They are one-way mathematical functions that are computationally infeasible to reverse-engineer. They take in input data, and output data of a fixed length.

#### An elementary comparison/explanation of what hashing is: Name Initials

Assume that in this case, we have a list of people's names. <br>
The names will be the input data, and we want the output data to be a fixed string of 3 letters.<br><b>
John F. Kennedy  &rarr;  JFK<br>
George W. Bush  &rarr;  GWB<br>
Franklin D. Roosevelt  &rarr;  FDR<br></b>
However, we know that in defining initials there is a rule where we take the first letter of the first, middle, and last names. This is an example of a one-way function; if someone were to ask you to guess the whole name only given three initials, say, GWB, you would most likely not be able to guess George W. Bush. <br>
Hashing functions work like this. However, in hashing, there is no rule or pattern from the input to output data. It is not computationally feasible to try to guess the exact input given an output hash. Take a look at the next example after the function we have defined.

In [2]:
def updatehash(*args):
    hashing_text = ''; h = sha256()
    for arg in args:
        hashing_text += str(arg)

    h.update(hashing_text.encode('utf-8'))
    return h.hexdigest()


In [3]:
updatehash("GWB")

'1355d8b4abf4f35eeec97303c7a0fa7087d8144d8905c09cb016912f8942e9c5'

In [4]:
updatehash("GWb")

'e7593b8d063b1ac11de6979fde149d89ea6dbe01a6bc7a6e48d2ce59c75b1cf5'

The two output hashes above are completely different. Even one small change, in this case, changing a letter to lowercase, results in a completely different hash.<br>
Hashing works with any form of input data. Here is an example of a hash of a list of strings. Notice that regardless of the size of the input data, we will always get a 64 digit sequence comprised of the numbers 0-9 and a-f. This is because the hashing algoriths follow a hexadecimal numbering system.

In [5]:
updatehash(['Hello', 'Goodbye'])

'c5fa9189ee592f7fd1d4ddd5169353ec08971ebdaead1ab0f560fe299c43433b'

### Constructing the Block Class

Using OOP, we create a new class called a Block. This will allow us to define certain variables and methods that we want the Block object to have. <br>
Every block in a blockchain contains certain elements of data. Because we are implementing a very simplified blockchain, I have only included the essential elements: <b>data, hash, nonce, previous hash</b>. <br>
The previous hash is the hash of the preceding block that was mined right before. Since this element is used in each block, this means that the previous block is associated with the next block. Since any change affects the output hash, this ensures that the blocks that have already been mined in the chain have not been tampered with.

In [6]:
class Block():
    data = None
    hash = None
    nonce = 0
    previous_hash = '0'* 64

    def __init__(self, data, number = 0):
        self.data = data
        self.number = number

    def hash(self):
        return updatehash(
            self.previous_hash,
            self.number,
            self.data,
            self.nonce
        )

    def __str__(self):
        return str('Block #: %s\nHash: %s\nPrevious Hash: %s\nData: %s\nNonce: %s\n' %(
            self.number,
            self.hash(),
            self.previous_hash,
            self.data,
            self.nonce
        ))

### Constructing the Blockchain Class

Again using OOP, we create a new class called a Blockchain. Here we have defined functions that will add, remove, and mine blocks to the chain. <br>
I have also defined a function that checks whether the block is valid (meaning that the data in the block has not been modified).

In [7]:
class Blockchain():

    def __init__(self, difficulty, chain = []):
        self.difficulty = difficulty
        self.chain = chain

    def add(self, block):
        self.chain.append(block)

    def remove(self, block):
        self.chain.remove(block)

    def mine(self, block):
        try:
            block.previous_hash = self.chain[-1].hash()
        except IndexError:
            pass
        while True:
            if block.hash()[:self.difficulty] == "0" * self.difficulty:
                self.add(block); break
            else:
                block.nonce += 1

    def isValid(self):
        for i in range(1, len(self.chain)):
            _previous = self.chain[i].previous_hash
            _current = self.chain[i-1].hash()
            if _previous != _current or _current[:self.difficulty] != '0'* self.difficulty:
                return False
        return True

### Defining the Proof-of-work function

This is essentially where the consensus algorithm comes in. Remember that earlier above in the Block class object there is an element called a <b>nonce</b>. The way PoW works is that the computer is changing an element of the block (the nonce) and recomputing a hash that meets the requirements of the consensus algorithm. Every piece of data in the block remains the same, but the nonce is just a number that is used incrementally to find a hash that satisfies the consensus algorithm rule. 

In [8]:
def mine(data, difficulty):
    blockchain = Blockchain(difficulty)

    num = 0
    for item in data:
        num += 1
        blockchain.mine(Block(item, num))

    for block in blockchain.chain:
        print(block)

In [9]:
blockchain1 = mine(["hi"], 2)

Block #: 1
Hash: 00721179dd91101323879a0df33f8a6a151842ad7c67303b3edfb7ef1249fa40
Previous Hash: 0000000000000000000000000000000000000000000000000000000000000000
Data: hi
Nonce: 240



In [10]:
blockchain1 = mine(["bye"], 3)

Block #: 1
Hash: 00721179dd91101323879a0df33f8a6a151842ad7c67303b3edfb7ef1249fa40
Previous Hash: 0000000000000000000000000000000000000000000000000000000000000000
Data: hi
Nonce: 240

Block #: 1
Hash: 000b93934a91e5b892d3629e2ca7aec1f5c55018c3bd3eaf9b4f594d34e76f65
Previous Hash: 00721179dd91101323879a0df33f8a6a151842ad7c67303b3edfb7ef1249fa40
Data: bye
Nonce: 3038



### Understanding the above mining process

First, we mined the data "hi" to the blockchain with a difficulty of 2. Notice that the Nonce is 240. This means that the computer had to recompute a new hash 240 times until it found one that satisfied the requirements we set above. In the next example, we mined some new data ("bye") to the chain but we increased the difficulty--so the computer had to recompute a hash 3038 times. Mining data to the chain with a higher difficulty also means that the latency time incrases. This is why you may have heard that Proof-of-Work is computationally expensive and takes up large amounts of resources. In a real blockchain, the consensus algorithms are much more strict.